In [2]:
import os
import re
import random
import glob 
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [30]:
train_list=os.listdir("S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train")
#test_set=os.listdir("S://eDU//Top Docs//DL R&D Projects//Research papers for DRDO_AI_Challenge//For Dataset//DukeMTMC-reID//DukeMTMC-reID//bounding_box_test")

l=len(train_list)
print(l)

traindf=pd.DataFrame(train_list)

train_label=[]
for n in train_list:
    arr=re.split(r'_',n)
    train_label.append(arr[0])

    
traindf['PersonID']=train_label
traindf.columns= ['ImageName','PersonID']

print(traindf)

traindf.to_csv('trainPersonID.csv', index=True)


classes = list(set(train_label)) 
print(len(classes))
print(classes)

16522
                  ImageName PersonID
0      0001_c2_f0046182.jpg     0001
1      0001_c2_f0046302.jpg     0001
2      0001_c2_f0046422.jpg     0001
3      0001_c2_f0046542.jpg     0001
4      0001_c2_f0046662.jpg     0001
...                     ...      ...
16517  7140_c4_f0175508.jpg     7140
16518  7140_c4_f0175628.jpg     7140
16519  7140_c4_f0175748.jpg     7140
16520  7140_c4_f0175868.jpg     7140
16521  7140_c4_f0175988.jpg     7140

[16522 rows x 2 columns]
702
['0368', '0209', '0252', '0108', '0081', '0744', '0528', '2432', '0286', '0166', '0124', '4243', '4448', '0614', '0228', '1239', '0310', '0133', '4618', '4602', '0208', '4336', '0406', '0320', '0724', '1259', '1696', '0520', '0148', '0639', '0799', '1242', '0161', '0821', '0665', '0682', '4140', '4277', '4238', '0284', '0401', '3765', '4811', '0883', '2742', '4796', '0407', '4063', '0282', '0246', '0131', '4751', '4292', '0831', '4502', '0478', '0417', '0692', '3582', '0687', '0152', '1246', '2422', '1363', '0037',

In [3]:
base_model = tf.keras.applications.ResNet50(input_shape=(224,224,3),weights='imagenet', include_top = False)
print(base_model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [13]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
headModel = tf.keras.layers.Dense(702, activation ='softmax')(x)

In [14]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=headModel)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [15]:
for i, layer in enumerate(model.layers):
  print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [18]:
for layer in model.layers[:175]:
  layer.trainable = False

In [19]:
for layer in model.layers[175:]:
  layer.trainable = True

In [31]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

img_size=(300,300)

train_files=glob.glob('S:/eDU/TopDocs/DLProjects/DRDO/Dataset/DukeMTMC-reID/DukeMTMC-reID/bounding_box_train')

train_imgs=[image.img_to_array(image.load_img(img,target_size=img_size))for img in train_files]

train_imgs.size

train_imgs=np.array(train_imgs)